In [ ]:
# Install required libraries
!pip install --upgrade diffusers transformers torch accelerate imageio-ffmpeg

import torch
import gc
import imageio
import numpy as np
from diffusers import DiffusionPipeline, StableDiffusionPipeline
from PIL import Image
from IPython.display import display, HTML
from base64 import b64encode
import os

# Clear memory
torch.cuda.empty_cache()
gc.collect()

# Define prompt
text_input = "A beautiful sunset over the mountains with vibrant colors"
output_file = "generated_video.mp4"
num_frames = 24  # 3 seconds at 8 FPS
fps = 8

# Step 1: Try HotShot-XL for text-to-video on CPU
try:
    # Initialize HotShot-XL pipeline
    pipe = DiffusionPipeline.from_pretrained(
        "hotshotco/hotshot-xl",
        torch_dtype=torch.float32,  # Use float32 for CPU
        use_safetensors=True
    )

    # Optimize pipeline (CPU-based)
    pipe.unet.enable_gradient_checkpointing()  # Save memory

    # Generate video
    video_frames = pipe(
        prompt=text_input,
        num_frames=num_frames,
        height=256,
        width=256,
        num_inference_steps=8,
        guidance_scale=7.5,
        negative_prompt="blur, low quality",
        generator=torch.Generator().manual_seed(42),
        fps=fps
    ).frames[0]

    # Clear memory
    del pipe
    gc.collect()

    # Export video
    imageio.mimwrite(output_file, video_frames, fps=fps)

except Exception as e:
    print(f"HotShot-XL failed: {str(e)}. Falling back to static video with Stable Diffusion.")

    # Step 2: Fallback to Stable Diffusion v1-5 for single image on CPU
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float32,  # Use float32 for CPU
        use_safetensors=True
    )

    # Optimize pipeline
    pipe.unet.enable_gradient_checkpointing()

    # Generate image
    image = pipe(
        prompt=text_input,
        height=256,
        width=256,
        num_inference_steps=8,
        guidance_scale=7.5,
        negative_prompt="blur, low quality",
        generator=torch.Generator().manual_seed(42)
    ).images[0]

    # Save image
    image.save("start_image.png")
    del pipe
    gc.collect()

    # Create static video by looping image
    image_np = np.array(Image.open("start_image.png"))
    writer = imageio.get_writer(output_file, fps=fps)
    for _ in range(num_frames):
        writer.append_data(image_np)
    writer.close()

# Step 3: Display video inline in Colab
def display_video_from_file(video_file):
    with open(video_file, "rb") as f:
        video_data = f.read()
    video_b64 = b64encode(video_data).decode()
    video_html = f"""
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
    </video>
    """
    display(HTML(video_html))

display_video_from_file(output_file)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 841.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

HotShot-XL failed: unet/hotshot_xl.py as defined in `model_index.json` does not exist in hotshotco/hotshot-xl and is not a module in 'diffusers/pipelines'.. Falling back to static video with Stable Diffusion.


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
